In [ ]:
import pandas as pd 
import torch
import torch.nn as nn
import transformers
import json
from datasets import load_dataset
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging,Trainer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

/home/samuel/.conda/envs/hw2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/samuel/.conda/envs/hw2/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= True,
)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

NameError: name 'BitsAndBytesConfig' is not defined

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.add_eos_token = True
tokenizer.add_bos_token=True
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [2]:
def formatting_text(data):
    # prompt = """[INST]根據所提供的文章内容，為每個问题选择一个符合文意且正确合理的答案。在回答时请"严格遵守"格式：在 '<answer>' 和 '</answer>' 之间填写答案，格式为 '問題題號 : 选项数字.正确答案'。范例 :<answer>問題1: 1.答案</answer> <answer>問題2: 1.答案</answer>..."""
    prompt = """###系统指令###
    你是一个专业的文章问题回答人员，你的任务是根据所提供的文章内容，为每个问题选择一个符合文意且正确合理的答案。如果你的回答是正確的，我会给你小费!"""
    # prompt = """
    # ###系統指令###
    # 请仔细阅读所提供的文章内容。根据文章的信息，为每个问题选择最合理且正确的答案。在回答每个问题时，你需要“严格遵守”以下的格式规定：

    # - 每个回答都应该放在 '<answer>' 和 '</answer>' 标签之间。
    # - 每个回答的格式应该是 '问题编号 : 选项数字.正确答案'。
    # - 确保每个问题的答案都是清晰并且独立的。

    # 例如，如果文章中的问题1的正确答案是选项1，问题2的正确答案是选项2，你应该这样回答：

    # <answer>问题1: 1.正确答案</answer>
    # <answer>问题2: 2.正确答案</answer>

    # 请确保按照这种格式准确回答每个问题，以便答案可以被清楚地识别和整理。
    # # """
    # prompt = """###系統指令###
    # 请仔细阅读所提供的文章内容。根据文章的信息，为每个问题选择最合理且正确的答案。在回答每个问题时，请按照以下范例回答：
    # "
    # ###答案###
    # 问题1: "
    # """

    paragraph = data['paragraph']
    all_questions = ""
    for i,q in enumerate(data["questions"]):
        question = f"问题{i+1}:\n{q['question']}"
        options = " ".join([f"{i+1}.{c}"for i,c in enumerate(q['choice'])])
        all_questions+= f"{question}\n选项:{options}\n"
    text= f"{prompt} \n\n###文章内容###\n{paragraph}\n###问题###\n{all_questions}[/INST]"
    return text
def formatting_label(data):
    all_answers = ""
    for i,q in enumerate(data["questions"]):
        a = q['choice']
        question = f"问题{i+1}:"
        answer = f"{a.index(q['answer'])+1}.{q['answer']}"
        all_answers+= f"{question} {answer}\n"
    text = f"\n###答案###\n{all_answers}"
    return text
def format_prompt(data):

    prompt = f"{data['text']}\n{data['label']}"
    return prompt
def read_json(mode):
    with open(f"dataset/{mode}.json", 'r') as file:
        data = json.load(file)
    data = pd.DataFrame(data)
    return data
train_data = read_json("train")
valid_data = read_json("val")
test_data = read_json("test")
train_data['text'] = train_data.apply(formatting_text,axis=1)
train_data['label'] = train_data.apply(formatting_label,axis=1)
train_data['prompt'] = train_data.apply(format_prompt,axis=1)
train_data = train_data[['text','label','prompt']]
valid_data['text'] = valid_data.apply(formatting_text,axis=1)
valid_data['label'] = valid_data.apply(formatting_label,axis=1)
valid_data['prompt'] = valid_data.apply(format_prompt,axis=1)
valid_data = valid_data[['text','label','prompt']]

In [3]:
print(train_data['prompt'][0])

###系统指令###
    你是一个专业的文章问题回答人员，你的任务是根据所提供的文章内容，为每个问题选择一个符合文意且正确合理的答案。如果你的回答是正確的，我会给你小费! 

###文章内容###
许多动物的某些器官感觉特别灵敏，它们能比人类提前知道一些灾害事件的发生，例如，海洋中的水母能预报风暴，老鼠能事先躲避矿井崩塌或有害气体，等等。地震往往能使一些动物的某些感觉器官受到刺激而发生异常反应。如一个地区的重力发生变异，某些动物可能通过它们的平衡器官感觉到；一种振动异常，某些动物的听觉器官也许能够察觉出来。地震前地下岩层早已在逐日缓慢活动，而断层面之间又具有强大的摩擦力。这种摩擦力会产生一种低于人的听觉所能感觉到的低频声波。人对每秒20次以上的声波才能感觉到，而动物则不然。那些感觉十分灵敏的动物，在感触到这种低声波时，便会惊恐万状，以至出现冬蛇出洞、鱼跃水面等异常现象。
###问题###
问题1:
动物的器官感觉与人的相比有什么不同?
选项:1.没有人的灵敏 2.和人的差不多 3.和人的一样好 4.比人的灵敏
问题2:
录音中提到能预报风暴的动物是什么?
选项:1.蛇 2.老鼠 3.水母 4.鱼
问题3:
低频声波至少要达到每秒多少次才能被人感觉到?
选项:1.20次 2.20次以上 3.20次以下 4.以上都对
问题4:
动物感觉到低频声波时会有怎样的表现?
选项:1.兴奋 2.逃跑 3.跳跃 4.害怕
[/INST]

###答案###
问题1:
动物的器官感觉与人的相比有什么不同?
答案: 4.比人的灵敏
问题2:
录音中提到能预报风暴的动物是什么?
答案: 3.水母
问题3:
低频声波至少要达到每秒多少次才能被人感觉到?
答案: 2.20次以上
问题4:
动物感觉到低频声波时会有怎样的表现?
答案: 4.害怕



In [55]:
# testing model output
def generate_response(prompt, model):
  tokenizer.padding_side = 'left'
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=500, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")
print(generate_response(train_data['text'][90],model))

/home/samuel/.conda/envs/hw2/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> </s>ria>問題1:  Shanlan's husband is a 2.  Shanlan's husband is a kind, simple, and responsible person.
<answer>問題2: Shanlan is often out because 1. because she has a job outside.
<answer>問題3:  On weekends, after getting up, "I" first do 3. go to the market to buy things.

<answer>
```
```</s>


In [6]:
train_data['prompt'].apply(len).describe()

count    3138.000000
mean      474.684512
std       314.046162
min       178.000000
25%       243.000000
50%       328.000000
75%       678.000000
max      1923.000000
Name: prompt, dtype: float64

In [8]:
def preprocess(data): #full prompt
    model_inputs = tokenizer(data['prompt'], padding="max_length", max_length=1940,truncation=True, return_tensors='pt')
    # label = tokenizer(data['label'], padding="max_length", max_length=50,truncation=True, return_tensors='pt')
    model_inputs["labels"] = model_inputs["input_ids"].clone()
    return model_inputs
new_train_data = datasets.Dataset.from_pandas(train_data,split='train')
new_valid_data = datasets.Dataset.from_pandas(valid_data,split='train')
tokenized_train_data = new_train_data.map(preprocess,remove_columns = new_train_data.column_names,batched=True)
tokenized_valid_data = new_valid_data.map(preprocess,remove_columns = new_valid_data.column_names,batched=True)
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


Map: 100%|██████████| 1046/1046 [00:00<00:00, 2921.09 examples/s]


In [9]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[ "q_proj",
        "k_proj",""
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",]
)
model = get_peft_model(model, peft_config)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(model)

trainable params: 85041152 || all params: 3837112320 || trainable%: 2.2162799758751914


In [11]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)
model = accelerator.prepare_model(model)

In [12]:
training_arguments = TrainingArguments(
    output_dir="./models/chat_model_finetune", #baseline_with_complete_prompt
    logging_dir="./logs/chat_model_finetune",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=100,
    save_total_limit = 4,
    logging_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    do_eval=True,
    lr_scheduler_type="constant",
    metric_for_best_model = "eval_loss",    
)
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_data,
    eval_dataset = tokenized_valid_data,
    tokenizer=tokenizer,
    args=training_arguments,
    data_collator=data_collator,
)
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=new_train_data,
#     eval_dataset = new_valid_data,
#     peft_config=peft_config,
#     max_seq_length= 1150,
#     dataset_text_field="prompt",
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing= False,
#     compute_metrics = compute_metrics
# )

In [14]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

## test

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.55s/it]


In [5]:
ft_model = PeftModel.from_pretrained(base_model, "./models/chat_model_finetune2/checkpoint-700")

In [6]:
test_data = read_json("test")
test_data['text'] = test_data.apply(formatting_text,axis=1)

In [8]:
print(test_data['text'][0])

###系统指令###
    你是一个专业的文章问题回答人员，你的任务是根据所提供的文章内容，为每个问题选择一个符合文意且正确合理的答案。如果你的回答是正確的，我会给你小费! 

###文章内容###
一个孕妇在路上行走。一个小女孩走过来问她：“阿姨，您的肚子为什么这么大?”孕妇说：“因为肚子里有孩子啊!”小女孩说：“阿姨，您是怕麻烦吧?”孕妇说：“啊?为什么?”小女孩说：“您嫌孩子抱着不方便，就把他放进肚子里了嘛。”
###问题###
问题1:
请选出与试题内容一致的一项。
选项:1.小女孩怕麻烦 2.小女孩认为孕妇怕麻烦 3.孕妇怕麻烦 4.孕妇很聪明
[/INST]


In [12]:
import re
from tqdm import tqdm, trange
from transformers import StoppingCriteriaList,StoppingCriteria
ft_model.eval()
predictions = []
class StopWords(StoppingCriteria):
    def __init__(self, tk, stop_words: list[str]):
        self.tk = tk
        self.stop_tokens = stop_words

    def __call__(self, input_ids, *_) -> bool:
        s = self.tk.batch_decode(input_ids)[0]
        for t in self.stop_tokens:
            if s.endswith(t):
                return True
        return False
sw = StopWords(tokenizer, ["</answer>"])
scl = StoppingCriteriaList([sw])
with torch.no_grad():
    for i in trange(len(test_data)):
        eval_prompt = test_data['text'][i]
        model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
        pred = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100,pad_token_id = tokenizer.eos_token_id)[0], skip_special_tokens=True)
        pred = pred.replace(eval_prompt,"")
        predictions.append(pred)
        # print(eval_prompt+"\n")
        # print(pred)
        # break


100%|██████████| 1045/1045 [55:00<00:00,  3.16s/it]


In [14]:
#從模型預測的結果中提取答案
import re
ans= []
# predictions = pd.read_csv("predictions.csv")['result']
for q, pred in zip(test_data['questions'],predictions):
    num_of_choice = len(q)
    text = pred
    # Updated regex pattern to extract "yes" or "no" after "### Answer:"
    # pattern_flexible = r'<answer>(.*?)</answer>'
    

    # Extracting the answer using the updated pattern
    # match_updated = re.search(pattern_flexible, text, re.IGNORECASE | re.DOTALL)
    
    
    # Extracting the answer
    # answer_updated = match_updated.group(1) if match_updated else "No match found"
    
    matches = re.findall(r'答案:\s*(\d+)', text)
    ans.extend(matches[:num_of_choice])

In [11]:
final_ans = [int(a.split(":")[1][1])-1 for a in ans]

[1,
 0,
 2,
 1,
 1,
 1,
 3,
 1,
 2,
 0,
 0,
 2,
 1,
 3,
 0,
 1,
 2,
 1,
 2,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 2,
 0,
 1,
 2,
 3,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 3,
 3,
 2,
 3,
 0,
 3,
 2,
 2,
 0,
 2,
 2,
 0,
 1,
 1,
 0,
 2,
 3,
 1,
 2,
 0,
 3,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 1,
 3,
 2,
 2,
 0,
 2,
 1,
 0,
 0,
 1,
 2,
 0,
 2,
 3,
 1,
 1,
 3,
 2,
 0,
 2,
 1,
 0,
 3,
 2,
 1,
 0,
 3,
 2,
 3,
 0,
 1,
 2,
 3,
 1,
 0,
 1,
 0,
 1,
 2,
 2,
 3,
 3,
 0,
 0,
 1,
 2,
 3,
 3,
 2,
 0,
 0,
 2,
 0,
 2,
 3,
 2,
 1,
 0,
 1,
 2,
 0,
 3,
 1,
 1,
 1,
 2,
 1,
 3,
 2,
 1,
 0,
 3,
 0,
 0,
 3,
 0,
 2,
 1,
 2,
 0,
 3,
 0,
 1,
 1,
 3,
 3,
 1,
 1,
 1,
 1,
 3,
 0,
 2,
 1,
 0,
 1,
 2,
 3,
 0,
 0,
 1,
 0,
 0,
 3,
 3,
 0,
 1,
 1,
 1,
 0,
 2,
 2,
 3,
 2,
 3,
 3,
 3,
 1,
 3,
 3,
 1,
 0,
 2,
 3,
 0,
 1,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 2,
 0,
 3,
 2,
 2,
 2,
 0,
 1,
 1,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 3,
 2,
 3,
 2,
 2,
 0,
 1,
 2,
 1,
 0,
 3,
 0,


In [35]:
pd.DataFrame({"answer":final_ans},index=range(8005, 8005 + len(final_ans))).to_csv("submission.csv",index_label="index")